# 15: Nextbike data part 2 - get which Nextbike trips started/ended at stations

In this notebook, we will look, which Nextbike trips started within one of our stop buffer zones. If a trip started (or ended or both) in such a buffer zone, the information will be added to the Nextbike dataset
- start station 
    - id
    - name
    - buffer zone
- end station
    - id
    - name
    - buffer zone

In [ ]:
Note: Originally on local machine performed

In [47]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
from shapely import Point, wkb, wkt

In [48]:
data_month_single = '7'
data_month_double = '07'

In [49]:
# load nextbike trip data
nextbike = pd.DataFrame(pd.read_parquet("../Data/14_nextbike_part1.parquet", engine="pyarrow"))
nextbike

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,start_location,end_location,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters
215356,21395440,2023-01-01 00:01:00,2023-01-01 00:26:00,False,False,b'\x01\x01\x00\x00\x00\xf0\xa7\xc6K7aI@\xee\xc...,b'\x01\x01\x00\x00\x00ca\x88\x9c\xbe^I@\xb7\xd...,bike,0 days 00:25:00,2.408602,2408.601747
215357,21395441,2023-01-01 00:01:00,2023-01-01 00:16:00,False,False,b'\x01\x01\x00\x00\x00}ZE\x7fh^I@\xd8G\xa7\xae...,b'\x01\x01\x00\x00\x00\xda\xac\xfa\\m_I@o\xf2[...,bike,0 days 00:15:00,2.122154,2122.154442
215371,21413775,2023-01-04 22:56:00,2023-01-04 23:11:00,False,False,b'\x01\x01\x00\x00\x00\xbct\x93\x18\x04ZI@B\x0...,b'\x01\x01\x00\x00\x00Xp?\xe0\x81YI@\xe2\x05\x...,bike,0 days 00:15:00,1.117285,1117.284795
215373,21413777,2023-01-04 23:01:00,2023-01-04 23:21:00,False,False,b'\x01\x01\x00\x00\x00\x0c\x03\x96\\\xc5\\I@ \...,b'\x01\x01\x00\x00\x00e\xaa`TR[I@5\xb5l\xad/r\...,bike,0 days 00:20:00,4.812307,4812.306771
215374,21413778,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,b'\x01\x01\x00\x00\x00\x92\x91\xb3\xb0\xa7]I@2...,b'\x01\x01\x00\x00\x00\x1b\x9e^)\xcb\\I@/\xc0>...,bike,0 days 00:10:00,0.756691,756.691454
...,...,...,...,...,...,...,...,...,...,...,...
2899324,53247874,2023-12-15 15:16:00,2023-12-15 15:26:00,False,True,b'\x01\x01\x00\x00\x00\xe8\x9f\xe0bEYI@\xdc.4\...,b'\x01\x01\x00\x00\x00;\xfb\xca\x83\xf4ZI@L\xf...,bike,0 days 00:10:00,1.476760,1476.759686
2899327,53247875,2023-12-15 15:31:00,2023-12-15 15:36:00,True,False,b'\x01\x01\x00\x00\x00;\xfb\xca\x83\xf4ZI@L\xf...,b'\x01\x01\x00\x00\x00\x88\x0e\x81#\x81ZI@\xb2...,bike,0 days 00:05:00,0.494814,494.813946
2899336,53247878,2023-12-15 17:41:00,2023-12-15 17:51:00,False,True,b'\x01\x01\x00\x00\x00\xf3\x90)\x1f\x82ZI@Q\x1...,b'\x01\x01\x00\x00\x00;\xfb\xca\x83\xf4ZI@L\xf...,bike,0 days 00:10:00,0.491459,491.458771
2899339,53247879,2023-12-15 18:16:00,2023-12-15 18:31:00,True,False,b'\x01\x01\x00\x00\x00;\xfb\xca\x83\xf4ZI@L\xf...,b'\x01\x01\x00\x00\x00\x8b\xff;\xa2BYI@\x90\x8...,bike,0 days 00:15:00,1.461322,1461.321822


In [50]:
# transform geometry
point_start = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(nextbike["start_location"], crs=4326))
point_end = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(nextbike["end_location"], crs=4326))

In [51]:
# drop geometry columns in wrong format
nextbike = nextbike.drop(columns=["start_location", "end_location"])

In [52]:
# replace geometry columns with correct format
nextbike['start_location'] = point_start
nextbike['end_location'] = point_end

In [53]:
nextbike

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location
215356,21395440,2023-01-01 00:01:00,2023-01-01 00:26:00,False,False,bike,0 days 00:25:00,2.408602,2408.601747,POINT (50.75950 7.12540),POINT (50.74019 7.11529)
215357,21395441,2023-01-01 00:01:00,2023-01-01 00:16:00,False,False,bike,0 days 00:15:00,2.122154,2122.154442,POINT (50.73756 7.08446),POINT (50.74553 7.06709)
215371,21413775,2023-01-04 22:56:00,2023-01-04 23:11:00,False,False,bike,0 days 00:15:00,1.117285,1117.284795,POINT (50.70325 7.15425),POINT (50.69928 7.16349)
215373,21413777,2023-01-04 23:01:00,2023-01-04 23:21:00,False,False,bike,0 days 00:20:00,4.812307,4812.306771,POINT (50.72477 7.06972),POINT (50.71345 7.11151)
215374,21413778,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,bike,0 days 00:10:00,0.756691,756.691454,POINT (50.73168 7.10039),POINT (50.72495 7.09908)
...,...,...,...,...,...,...,...,...,...,...,...
2899324,53247874,2023-12-15 15:16:00,2023-12-15 15:26:00,False,True,bike,0 days 00:10:00,1.476760,1476.759686,POINT (50.69743 7.12931),POINT (50.71059 7.12687)
2899327,53247875,2023-12-15 15:31:00,2023-12-15 15:36:00,True,False,bike,0 days 00:05:00,0.494814,494.813946,POINT (50.71059 7.12687),POINT (50.70707 7.12412)
2899336,53247878,2023-12-15 17:41:00,2023-12-15 17:51:00,False,True,bike,0 days 00:10:00,0.491459,491.458771,POINT (50.70710 7.12413),POINT (50.71059 7.12687)
2899339,53247879,2023-12-15 18:16:00,2023-12-15 18:31:00,True,False,bike,0 days 00:15:00,1.461322,1461.321822,POINT (50.71059 7.12687),POINT (50.69735 7.12717)


In [54]:
nextbike['start_location'] = nextbike['start_location'].apply(lambda point: Point(point.y, point.x))
nextbike['end_location'] = nextbike['end_location'].apply(lambda point: Point(point.y, point.x))

In [55]:
nextbike

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location
215356,21395440,2023-01-01 00:01:00,2023-01-01 00:26:00,False,False,bike,0 days 00:25:00,2.408602,2408.601747,POINT (7.125399 50.7595),POINT (7.115293 50.740192)
215357,21395441,2023-01-01 00:01:00,2023-01-01 00:16:00,False,False,bike,0 days 00:15:00,2.122154,2122.154442,POINT (7.08446 50.737564),POINT (7.067087 50.745525)
215371,21413775,2023-01-04 22:56:00,2023-01-04 23:11:00,False,False,bike,0 days 00:15:00,1.117285,1117.284795,POINT (7.154247 50.70325),POINT (7.163489 50.699276)
215373,21413777,2023-01-04 23:01:00,2023-01-04 23:21:00,False,False,bike,0 days 00:20:00,4.812307,4812.306771,POINT (7.069716 50.724773),POINT (7.11151 50.71345)
215374,21413778,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,bike,0 days 00:10:00,0.756691,756.691454,POINT (7.100387 50.73168),POINT (7.09908 50.72495)
...,...,...,...,...,...,...,...,...,...,...,...
2899324,53247874,2023-12-15 15:16:00,2023-12-15 15:26:00,False,True,bike,0 days 00:10:00,1.476760,1476.759686,POINT (7.12931 50.69743),POINT (7.126873 50.710587)
2899327,53247875,2023-12-15 15:31:00,2023-12-15 15:36:00,True,False,bike,0 days 00:05:00,0.494814,494.813946,POINT (7.126873 50.710587),POINT (7.124117 50.707066)
2899336,53247878,2023-12-15 17:41:00,2023-12-15 17:51:00,False,True,bike,0 days 00:10:00,0.491459,491.458771,POINT (7.124128 50.707096),POINT (7.126873 50.710587)
2899339,53247879,2023-12-15 18:16:00,2023-12-15 18:31:00,True,False,bike,0 days 00:15:00,1.461322,1461.321822,POINT (7.126873 50.710587),POINT (7.127167 50.697346)


In [56]:
nextbike.info()

<class 'pandas.core.frame.DataFrame'>
Index: 771657 entries, 215356 to 2899345
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype          
---  ------                           --------------   -----          
 0   nextbike_trips_id                771657 non-null  int64          
 1   nextbike_trips_start_time        771657 non-null  datetime64[ns] 
 2   nextbike_trips_end_time          771657 non-null  datetime64[ns] 
 3   nextbike_trips_start_at_station  771657 non-null  bool           
 4   nextbike_trips_end_at_station    771657 non-null  bool           
 5   vehicle type                     771657 non-null  object         
 6   trip_duration                    771657 non-null  timedelta64[ns]
 7   trip_distance_kilometers         771657 non-null  float64        
 8   trip_distance_meters             771657 non-null  float64        
 9   start_location                   771657 non-null  object         
 10  end_location                   

In [57]:
nextbike = gpd.GeoDataFrame(nextbike, geometry='start_location')

In [58]:
nextbike = gpd.GeoDataFrame(nextbike, geometry='end_location')

In [59]:
nextbike = gpd.GeoDataFrame(nextbike, geometry='start_location')

In [60]:
nextbike.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 771657 entries, 215356 to 2899345
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype          
---  ------                           --------------   -----          
 0   nextbike_trips_id                771657 non-null  int64          
 1   nextbike_trips_start_time        771657 non-null  datetime64[ns] 
 2   nextbike_trips_end_time          771657 non-null  datetime64[ns] 
 3   nextbike_trips_start_at_station  771657 non-null  bool           
 4   nextbike_trips_end_at_station    771657 non-null  bool           
 5   vehicle type                     771657 non-null  object         
 6   trip_duration                    771657 non-null  timedelta64[ns]
 7   trip_distance_kilometers         771657 non-null  float64        
 8   trip_distance_meters             771657 non-null  float64        
 9   start_location                   771657 non-null  geometry       
 10  end_location           

In [61]:
nextbike

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location
215356,21395440,2023-01-01 00:01:00,2023-01-01 00:26:00,False,False,bike,0 days 00:25:00,2.408602,2408.601747,POINT (7.12540 50.75950),POINT (7.11529 50.74019)
215357,21395441,2023-01-01 00:01:00,2023-01-01 00:16:00,False,False,bike,0 days 00:15:00,2.122154,2122.154442,POINT (7.08446 50.73756),POINT (7.06709 50.74553)
215371,21413775,2023-01-04 22:56:00,2023-01-04 23:11:00,False,False,bike,0 days 00:15:00,1.117285,1117.284795,POINT (7.15425 50.70325),POINT (7.16349 50.69928)
215373,21413777,2023-01-04 23:01:00,2023-01-04 23:21:00,False,False,bike,0 days 00:20:00,4.812307,4812.306771,POINT (7.06972 50.72477),POINT (7.11151 50.71345)
215374,21413778,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,bike,0 days 00:10:00,0.756691,756.691454,POINT (7.10039 50.73168),POINT (7.09908 50.72495)
...,...,...,...,...,...,...,...,...,...,...,...
2899324,53247874,2023-12-15 15:16:00,2023-12-15 15:26:00,False,True,bike,0 days 00:10:00,1.476760,1476.759686,POINT (7.12931 50.69743),POINT (7.12687 50.71059)
2899327,53247875,2023-12-15 15:31:00,2023-12-15 15:36:00,True,False,bike,0 days 00:05:00,0.494814,494.813946,POINT (7.12687 50.71059),POINT (7.12412 50.70707)
2899336,53247878,2023-12-15 17:41:00,2023-12-15 17:51:00,False,True,bike,0 days 00:10:00,0.491459,491.458771,POINT (7.12413 50.70710),POINT (7.12687 50.71059)
2899339,53247879,2023-12-15 18:16:00,2023-12-15 18:31:00,True,False,bike,0 days 00:15:00,1.461322,1461.321822,POINT (7.12687 50.71059),POINT (7.12717 50.69735)


In [62]:
#vrs_data = pd.DataFrame(pd.read_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/11_data_clean.parquet", engine="pyarrow"))

In [63]:
vrs_data = pd.DataFrame(pd.read_parquet("../Data/10_VRS_data_clean.parquet", engine="pyarrow"))
#vrs_data = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/19_Merged_with_Weather_Holiday.csv"))
#vrs_data = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/10_VRS_data_clean.csv"))

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_18839/83190647.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  vrs_data = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/11_data_clean.csv"))


In [64]:
# transform geometry
vrs_data['station_point'] = vrs_data['station_point'].apply(wkt.loads)

In [65]:
vrs_data['buffer_zone'] = shapely.wkt.loads(vrs_data['buffer_zone'])

In [66]:
vrs_data = gpd.GeoDataFrame(vrs_data, geometry='station_point')

In [67]:
# transform geometry
#point = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(vrs_data["station_point"], crs=4326))
#buffer = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(vrs_data["buffer_zone"], crs=4326))

In [68]:
# drop geometry columns in wrong format
#vrs_data= vrs_data.drop(columns=["station_point", "buffer_zone"])

In [69]:
# replace geometry columns with correct format
#vrs_data['station_point'] = point
#vrs_data['buffer_zone'] = buffer

In [70]:
vrs_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 873991 entries, 0 to 873990
Data columns (total 31 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   Unnamed: 0.2              873991 non-null  int64   
 1   Unnamed: 0.1              873991 non-null  int64   
 2   Unnamed: 0                873991 non-null  int64   
 3   route_id                  873991 non-null  int64   
 4   agency_id                 873991 non-null  int64   
 5   route_short_name          873991 non-null  object  
 6   route_type                873991 non-null  int64   
 7   route_type_name           873991 non-null  object  
 8   agency_name               873991 non-null  object  
 9   service_id                873991 non-null  int64   
 10  trip_id                   873991 non-null  object  
 11  trip_headsign             873991 non-null  object  
 12  direction_id              873991 non-null  int64   
 13  shape_id             

## Check if Tier trip start and/or ends within a buffer zone

In [71]:
unique_stops = vrs_data[['stop_id', 'stop_name', 'buffer_zone']].drop_duplicates()

In [72]:
# check if trip starts in a buffer zone

# Create empty columns in tier_data
nextbike['start_stop_id'] = np.nan
nextbike['start_stop_name'] = np.nan
nextbike['start_buffer_zone'] = np.nan

# Iterate over each row in tier_data
for index, row in nextbike.iterrows():
    # Get the coordinates of tier_trips_end
    tier_trips_end_coords = row['start_location']	
    
    # Check if tier_trips_end coordinates lie within a buffer_zone in unique_stops
    for stop_index, stop_row in unique_stops.iterrows():
        buffer_zone = stop_row['buffer_zone']
        if buffer_zone.contains(tier_trips_end_coords):
            # If true, add the stop_id, stop_name, buffer_zone to the row in tier_data
            nextbike.at[index, 'start_stop_id'] = stop_row['stop_id']
            nextbike.at[index, 'start_stop_name'] = stop_row['stop_name']
            nextbike.at[index, 'start_buffer_zone'] = buffer_zone
            break

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_18839/155758629.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Bonn Eifelstr./Macke-Haus' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nextbike.at[index, 'start_stop_name'] = stop_row['stop_name']
/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_18839/155758629.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'POLYGON ((7.087229029184838 50.73780987711222, 7.08726750220833 50.7376371502933, 7.087279110330667 50.73746286218859, 7.087263742061531 50.73728869131959, 7.087221545714431 50.73711631505936, 7.087152927966523 50.73694739347796, 7.087058549930418 50.73678355335544, 7.086939320776297 50.73662637251606, 7.086796388966219 50.73647736463473, 7.086631131185321 50.73633796466191, 7.086445139076932 50

In [73]:
# check if trip ends in a buffer zone

# Create empty columns in tier_data
nextbike['end_stop_id'] = np.nan
nextbike['end_stop_name'] = np.nan
nextbike['end_buffer_zone'] = np.nan

# Iterate over each row in tier_data
for index, row in nextbike.iterrows():
    # Get the coordinates of tier_trips_end
    tier_trips_end_coords = row['end_location']	
    
    # Check if tier_trips_end coordinates lie within a buffer_zone in unique_stops
    for stop_index, stop_row in unique_stops.iterrows():
        buffer_zone = stop_row['buffer_zone']
        if buffer_zone.contains(tier_trips_end_coords):
            # If true, add the stop_id, stop_name, buffer_zone to the row in tier_data
            nextbike.at[index, 'end_stop_id'] = stop_row['stop_id']
            nextbike.at[index, 'end_stop_name'] = stop_row['stop_name']
            nextbike.at[index, 'end_buffer_zone'] = buffer_zone
            break

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_18839/965310777.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Bonn Konrad-Adenauer-Platz' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nextbike.at[index, 'end_stop_name'] = stop_row['stop_name']
/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_18839/965310777.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'POLYGON ((7.120193524138334 50.73956510756546, 7.120231877857076 50.73939236011865, 7.120243363058693 50.73921805899604, 7.120227869436752 50.7390438828444, 7.12018554651201 50.73887150908747, 7.120116802180585 50.73870259777108, 7.120022298774026 50.73853877557632, 7.119902946669738 50.73838162015463, 7.119759895513612 50.73823264493605, 7.119594523139856 50.73809328455641, 7.119408422295066 50.

In [74]:
nextbike

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone
215356,21395440,2023-01-01 00:01:00,2023-01-01 00:26:00,False,False,bike,0 days 00:25:00,2.408602,2408.601747,POINT (7.12540 50.75950),POINT (7.11529 50.74019),NaN,NaN,NaN,1500.0,Bonn Konrad-Adenauer-Platz,"POLYGON ((7.120193524138334 50.73956510756546,..."
215357,21395441,2023-01-01 00:01:00,2023-01-01 00:16:00,False,False,bike,0 days 00:15:00,2.122154,2122.154442,POINT (7.08446 50.73756),POINT (7.06709 50.74553),1194.0,Bonn Eifelstr./Macke-Haus,"POLYGON ((7.087229029184838 50.73780987711222,...",1432.0,Bonn Magdeburger Str.,"POLYGON ((7.068940359556346 50.74716635280296,..."
215371,21413775,2023-01-04 22:56:00,2023-01-04 23:11:00,False,False,bike,0 days 00:15:00,1.117285,1117.284795,POINT (7.15425 50.70325),POINT (7.16349 50.69928),NaN,NaN,NaN,NaN,NaN,NaN
215373,21413777,2023-01-04 23:01:00,2023-01-04 23:21:00,False,False,bike,0 days 00:20:00,4.812307,4812.306771,POINT (7.06972 50.72477),POINT (7.11151 50.71345),NaN,NaN,NaN,NaN,NaN,NaN
215374,21413778,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,bike,0 days 00:10:00,0.756691,756.691454,POINT (7.10039 50.73168),POINT (7.09908 50.72495),686.0,Bonn Universität/Markt,"POLYGON ((7.104885938336375 50.73267243628678,...",1134.0,Bonn Wilhelm-Levison-Str.,"POLYGON ((7.10179543530159 50.72471047165823, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899324,53247874,2023-12-15 15:16:00,2023-12-15 15:26:00,False,True,bike,0 days 00:10:00,1.476760,1476.759686,POINT (7.12931 50.69743),POINT (7.12687 50.71059),NaN,NaN,NaN,691.0,Bonn Ollenhauerstr.,"POLYGON ((7.129312001116849 50.71170776127034,..."
2899327,53247875,2023-12-15 15:31:00,2023-12-15 15:36:00,True,False,bike,0 days 00:05:00,0.494814,494.813946,POINT (7.12687 50.71059),POINT (7.12412 50.70707),691.0,Bonn Ollenhauerstr.,"POLYGON ((7.129312001116849 50.71170776127034,...",NaN,NaN,NaN
2899336,53247878,2023-12-15 17:41:00,2023-12-15 17:51:00,False,True,bike,0 days 00:10:00,0.491459,491.458771,POINT (7.12413 50.70710),POINT (7.12687 50.71059),NaN,NaN,NaN,691.0,Bonn Ollenhauerstr.,"POLYGON ((7.129312001116849 50.71170776127034,..."
2899339,53247879,2023-12-15 18:16:00,2023-12-15 18:31:00,True,False,bike,0 days 00:15:00,1.461322,1461.321822,POINT (7.12687 50.71059),POINT (7.12717 50.69735),691.0,Bonn Ollenhauerstr.,"POLYGON ((7.129312001116849 50.71170776127034,...",NaN,NaN,NaN


In [75]:
nextbike.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 771657 entries, 215356 to 2899345
Data columns (total 17 columns):
 #   Column                           Non-Null Count   Dtype          
---  ------                           --------------   -----          
 0   nextbike_trips_id                771657 non-null  int64          
 1   nextbike_trips_start_time        771657 non-null  datetime64[ns] 
 2   nextbike_trips_end_time          771657 non-null  datetime64[ns] 
 3   nextbike_trips_start_at_station  771657 non-null  bool           
 4   nextbike_trips_end_at_station    771657 non-null  bool           
 5   vehicle type                     771657 non-null  object         
 6   trip_duration                    771657 non-null  timedelta64[ns]
 7   trip_distance_kilometers         771657 non-null  float64        
 8   trip_distance_meters             771657 non-null  float64        
 9   start_location                   771657 non-null  geometry       
 10  end_location           

In [76]:
gdf = gpd.GeoDataFrame(nextbike, geometry='start_buffer_zone')

In [77]:
gdf = gpd.GeoDataFrame(nextbike, geometry='end_buffer_zone')

In [78]:
gdf = gpd.GeoDataFrame(nextbike, geometry='start_location')

In [79]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 771657 entries, 215356 to 2899345
Data columns (total 17 columns):
 #   Column                           Non-Null Count   Dtype          
---  ------                           --------------   -----          
 0   nextbike_trips_id                771657 non-null  int64          
 1   nextbike_trips_start_time        771657 non-null  datetime64[ns] 
 2   nextbike_trips_end_time          771657 non-null  datetime64[ns] 
 3   nextbike_trips_start_at_station  771657 non-null  bool           
 4   nextbike_trips_end_at_station    771657 non-null  bool           
 5   vehicle type                     771657 non-null  object         
 6   trip_duration                    771657 non-null  timedelta64[ns]
 7   trip_distance_kilometers         771657 non-null  float64        
 8   trip_distance_meters             771657 non-null  float64        
 9   start_location                   771657 non-null  geometry       
 10  end_location           

In [80]:

def parse_wkb(x):
    try:
        return wkb.loads(str(x), hex=True) if x else None
    except:
        return None

gdf['start_buffer_zone'] = gdf['start_buffer_zone'].apply(parse_wkb)


In [81]:
gdf

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone
215356,21395440,2023-01-01 00:01:00,2023-01-01 00:26:00,False,False,bike,0 days 00:25:00,2.408602,2408.601747,POINT (7.12540 50.75950),POINT (7.11529 50.74019),NaN,NaN,None,1500.0,Bonn Konrad-Adenauer-Platz,"POLYGON ((7.120193524138334 50.73956510756546,..."
215357,21395441,2023-01-01 00:01:00,2023-01-01 00:16:00,False,False,bike,0 days 00:15:00,2.122154,2122.154442,POINT (7.08446 50.73756),POINT (7.06709 50.74553),1194.0,Bonn Eifelstr./Macke-Haus,None,1432.0,Bonn Magdeburger Str.,"POLYGON ((7.068940359556346 50.74716635280296,..."
215371,21413775,2023-01-04 22:56:00,2023-01-04 23:11:00,False,False,bike,0 days 00:15:00,1.117285,1117.284795,POINT (7.15425 50.70325),POINT (7.16349 50.69928),NaN,NaN,None,NaN,NaN,NaN
215373,21413777,2023-01-04 23:01:00,2023-01-04 23:21:00,False,False,bike,0 days 00:20:00,4.812307,4812.306771,POINT (7.06972 50.72477),POINT (7.11151 50.71345),NaN,NaN,None,NaN,NaN,NaN
215374,21413778,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,bike,0 days 00:10:00,0.756691,756.691454,POINT (7.10039 50.73168),POINT (7.09908 50.72495),686.0,Bonn Universität/Markt,None,1134.0,Bonn Wilhelm-Levison-Str.,"POLYGON ((7.10179543530159 50.72471047165823, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899324,53247874,2023-12-15 15:16:00,2023-12-15 15:26:00,False,True,bike,0 days 00:10:00,1.476760,1476.759686,POINT (7.12931 50.69743),POINT (7.12687 50.71059),NaN,NaN,None,691.0,Bonn Ollenhauerstr.,"POLYGON ((7.129312001116849 50.71170776127034,..."
2899327,53247875,2023-12-15 15:31:00,2023-12-15 15:36:00,True,False,bike,0 days 00:05:00,0.494814,494.813946,POINT (7.12687 50.71059),POINT (7.12412 50.70707),691.0,Bonn Ollenhauerstr.,None,NaN,NaN,NaN
2899336,53247878,2023-12-15 17:41:00,2023-12-15 17:51:00,False,True,bike,0 days 00:10:00,0.491459,491.458771,POINT (7.12413 50.70710),POINT (7.12687 50.71059),NaN,NaN,None,691.0,Bonn Ollenhauerstr.,"POLYGON ((7.129312001116849 50.71170776127034,..."
2899339,53247879,2023-12-15 18:16:00,2023-12-15 18:31:00,True,False,bike,0 days 00:15:00,1.461322,1461.321822,POINT (7.12687 50.71059),POINT (7.12717 50.69735),691.0,Bonn Ollenhauerstr.,None,NaN,NaN,NaN


In [82]:
#nextbike['start_buffer_zone'] = nextbike['start_buffer_zone'].apply(lambda x: x.wkb if isinstance(x, gpd.GeoSeries) else None)



In [83]:
#nextbike['end_buffer_zone'] = nextbike['end_buffer_zone'].apply(lambda x: x.wkb if isinstance(x, gpd.GeoSeries) else None)

In [84]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 771657 entries, 215356 to 2899345
Data columns (total 17 columns):
 #   Column                           Non-Null Count   Dtype          
---  ------                           --------------   -----          
 0   nextbike_trips_id                771657 non-null  int64          
 1   nextbike_trips_start_time        771657 non-null  datetime64[ns] 
 2   nextbike_trips_end_time          771657 non-null  datetime64[ns] 
 3   nextbike_trips_start_at_station  771657 non-null  bool           
 4   nextbike_trips_end_at_station    771657 non-null  bool           
 5   vehicle type                     771657 non-null  object         
 6   trip_duration                    771657 non-null  timedelta64[ns]
 7   trip_distance_kilometers         771657 non-null  float64        
 8   trip_distance_meters             771657 non-null  float64        
 9   start_location                   771657 non-null  geometry       
 10  end_location           

In [85]:
nextbike.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 771657 entries, 215356 to 2899345
Data columns (total 17 columns):
 #   Column                           Non-Null Count   Dtype          
---  ------                           --------------   -----          
 0   nextbike_trips_id                771657 non-null  int64          
 1   nextbike_trips_start_time        771657 non-null  datetime64[ns] 
 2   nextbike_trips_end_time          771657 non-null  datetime64[ns] 
 3   nextbike_trips_start_at_station  771657 non-null  bool           
 4   nextbike_trips_end_at_station    771657 non-null  bool           
 5   vehicle type                     771657 non-null  object         
 6   trip_duration                    771657 non-null  timedelta64[ns]
 7   trip_distance_kilometers         771657 non-null  float64        
 8   trip_distance_meters             771657 non-null  float64        
 9   start_location                   771657 non-null  geometry       
 10  end_location           

In [86]:
nextbike['start_buffer_zone'] = str(nextbike['start_buffer_zone'])
nextbike['end_buffer_zone'] = str(nextbike['end_buffer_zone'])

In [87]:
nextbike.to_parquet("../Data/15_nextbike_part2.parquet", engine="pyarrow")

In [88]:
nextbike_not_nan = nextbike[nextbike['start_stop_id'].notna()]
nextbike_not_nan

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone
215357,21395441,2023-01-01 00:01:00,2023-01-01 00:16:00,False,False,bike,0 days 00:15:00,2.122154,2122.154442,POINT (7.08446 50.73756),POINT (7.06709 50.74553),1194.0,Bonn Eifelstr./Macke-Haus,215356 ...,1432.0,Bonn Magdeburger Str.,215356 POLYGON ((7.120193524138334 50.7395...
215374,21413778,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,bike,0 days 00:10:00,0.756691,756.691454,POINT (7.10039 50.73168),POINT (7.09908 50.72495),686.0,Bonn Universität/Markt,215356 ...,1134.0,Bonn Wilhelm-Levison-Str.,215356 POLYGON ((7.120193524138334 50.7395...
215388,21413792,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,bike,0 days 00:10:00,0.596288,596.287809,POINT (7.09742 50.73665),POINT (7.10250 50.73492),1102.0,Bonn Friedensplatz,215356 ...,1103.0,Bonn Markt,215356 POLYGON ((7.120193524138334 50.7395...
215394,21413798,2023-01-04 23:06:00,2023-01-04 23:16:00,False,False,bike,0 days 00:10:00,0.843076,843.075548,POINT (7.09750 50.73649),POINT (7.09646 50.74405),1102.0,Bonn Friedensplatz,215356 ...,NaN,NaN,215356 POLYGON ((7.120193524138334 50.7395...
215404,21413807,2023-01-04 23:11:00,2023-01-04 23:31:00,False,False,bike,0 days 00:20:00,1.749380,1749.380473,POINT (7.08697 50.73343),POINT (7.07257 50.72707),1140.0,Bonn Mozartstr.,215356 ...,NaN,NaN,215356 POLYGON ((7.120193524138334 50.7395...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899189,53247837,2023-12-12 06:51:00,2023-12-12 07:01:00,True,False,bike,0 days 00:10:00,1.345842,1345.841787,POINT (7.15918 50.68314),POINT (7.15722 50.69518),161.0,Bonn Bad Godesberg Bf,215356 ...,NaN,NaN,215356 POLYGON ((7.120193524138334 50.7395...
2899262,53247857,2023-12-13 21:01:00,2023-12-13 21:11:00,False,False,bike,0 days 00:10:00,1.159259,1159.259363,POINT (7.10180 50.72589),POINT (7.11064 50.72030),1131.0,Bonn Weberstr.,215356 ...,NaN,NaN,215356 POLYGON ((7.120193524138334 50.7395...
2899282,53247866,2023-12-14 11:51:00,2023-12-14 12:01:00,False,False,bike,0 days 00:10:00,0.860385,860.385079,POINT (7.13793 50.70071),POINT (7.13019 50.70055),698.0,Bonn Max-Löbner-Str./Friesdorf,215356 ...,NaN,NaN,215356 POLYGON ((7.120193524138334 50.7395...
2899327,53247875,2023-12-15 15:31:00,2023-12-15 15:36:00,True,False,bike,0 days 00:05:00,0.494814,494.813946,POINT (7.12687 50.71059),POINT (7.12412 50.70707),691.0,Bonn Ollenhauerstr.,215356 ...,NaN,NaN,215356 POLYGON ((7.120193524138334 50.7395...
